# Project 2: Classification

This project asks you to perform various experiments with classification. The dataset we are using is a toy dataset for credit card fraud detection:

https://www.kaggle.com/datasets/shubhamjoshi2130of/abstract-data-set-for-credit-card-fraud-detection

You will write code and discussion texts into code and text cells in this notebook. 

If a block starts with TODO:, this means that you need to write something there. 

Some code had been written for you to guide the project. Don't change the already written code.

## Grading
The points add up to 40, that is 30 + 10 bonus points. While there is no difference between the regular and the bonus points, I recommend that you solve the problems labeled "BONUS" after you finished the other ones. 


In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk

## Setup for the project

Here we load the dataset, and create the training and test datasets as numpy arrays.

In [2]:
df = pd.read_csv("creditcard.csv",  true_values=["Y"], false_values=["N"])
pd.show_versions()
print(f"Number of rows {len(df.index)}")
print(f"The columns of the database {df.columns}")
df.value_counts("isFradulent")

FileNotFoundError: ignored

In [ ]:
xfields = [
    'Average Amount/transaction/day',
       'Transaction_amount', 'Is declined', 'Total Number of declines/day',
       'isForeignTransaction', 'isHighRiskCountry', 'Daily_chargeback_avg_amt',
       '6_month_avg_chbk_amt', '6-month_chbk_freq']

df_shuffled = df.sample(frac=1) # shuffle the rows
x = df_shuffled[xfields].to_numpy(dtype=np.float64)
y = df_shuffled["isFradulent"].to_numpy(dtype=np.float64)
# the training data is the first 2000 rows, after shuffled
training_data_x = x[:2000]
training_data_y = y[:2000]
# the test data is the remaining
test_data_x = x[2000:]
test_data_y = y[2000:]

In [ ]:
print("Run this to help you with what number goes with what field:")
for i, x in enumerate(xfields):
    print(f"{i} = {x}")

In [ ]:
#data experiments
#training data x setup like rows     column 0, row 0-8 (avg amount, transaction, etc), training data y would be the fraudulent or not
print(training_data_x[1999][0])
print(training_data_x[398][5])


print(training_data_x[31][5])
print(training_data_y[33])


## P1: Create an accuracy metric (7 pts)
Create a simple accuracy metric function which for a pair of ground truth values $y$ and estimates $\hat{y}$ (both of them arrays) calculates the accuracy of the estimate $\hat{y}$. For instance, if you pass y = [1, 0, 1] and 
yhat = [1, 1, 0], the loss function should return 0.3

In [ ]:
def accuracy(y, yhat):
    ## implement here
    y_accuracy = np.array(y)
    yhat_accuracy = np.array(yhat)

    size = y_accuracy.size

    compare = len(np.where(y_accuracy == yhat_accuracy)[0])

    result = compare/size
    return result


In [ ]:
# test your function here
acc = accuracy([1, 0, 1], [1, 1, 0])
print(f"Accuracy is {acc}") # should print 0.33...

## P2: Implement a majority classifier (7 pts)
This classifier will always return the most likely value. Training the classifier means determining what is the most likely value (regardless vhat value you pass to it). For instance, if more than half of the transactions are fraudulent, then you just return fraudulent always. 

In [ ]:
def classify_majority(x, theta):
    # whatever the value of x, we return the theta
    theta = theta
    return theta

# TODO: implement the train majority function
def train_majority(training_x, training_y):
    # this function will have to determine which is more likely to 
    # be the value of y, one (true) or zero (false)

    #convert floats from training y to integers
    int_array = np.rint(training_y).astype(int) 

    #finds tha majority value of y 
    majority = np.bincount(int_array).argmax()


    # C.N 1: if x equals a certain average value that produces the y value, make the threshhold. anything before is true, anything before is false 
    return majority

In [ ]:
# TODO: use the train_majority function to find the theta value for the training dataset
theta = 0
theta = train_majority(training_data_x, training_data_y)
print(theta)
# TODO: now use the theta value to create the test_data_yhat array which contains the classification for each test value 
test_data_yhat = np.full_like(test_data_y, theta)
# TODO: now calculate the accuracy of the classifier using the function implemented in P1, and print it out
print(accuracy(test_data_y, test_data_yhat))


TODO: Discuss here the performance of the majority classifier. Would this beat a classifier that just returns random values? 

No it would not due to how the random values have a chance to not be accurate at all. At least with the majority classifier it'll be semi accurcate due to its basis on factual data.


## P3: Implement a hand engineered classifier (8 pts)

Engineer by hand a classifier function that predicts whether  a transaction is  fraudulent or not. Your function should have a $\theta$ parameter which allows you to tweak it. 
The problem requires you to design a function that performs this classification, tweak its parameters, and measure its accuracy for the best parametrization you found. You should aim for a function that, at minimum, performs better than the majority classifier. 

In [ ]:
# TODO: implement here your hand-engineered classifier
# The example below is just a very bad example, but it gives you an idea of how you can reason about the classification problem.
# In your implementation, you should try to actually find some kind of clever algorithm. You can also use more complex parametrizations

def classify_handwritten(x, theta):
    """Fraudulent transaction classifier. In this test example, we classify every foreign transaction as fraudulent. Transactions larger than theta[0] are also fraudulent"""
    if x[5] == 1:
        return 1
    else:
        if x[4] == 1:
            return 1
    if x[1] > theta[0]:
        return 1
    return 0


def average_finder(training_data_x, training_data_y):
    #obtain all x's that is flagged as fraudluent
    #go through data and if training data y is marked as fraudulent, record its index of training data x add it to a np array
    af_y_array = np.rint(training_data_y).astype(int) 
    fradulentcounter = 0
    total = 0
    j = 0

    for i in af_y_array:
      if i == 1:
        #if training data is fraudulent, insert that row's average transaction into the average array so we can calculate the average
        fradulentcounter = fradulentcounter + 1
        total = total + training_data_x[j][0]
        j+=1
    # get the average value of the average transactions value
    average = total / fradulentcounter
    return average


def banaag_classifier(test_data_x, average, j):
    #if test data x average meets threshold or above, mark it is a 1, else 0
    comparer_bc = test_data_x[j][0]

    #print("Comparer_BC:", comparer_bc)
    #print("Average:",average)
    if comparer_bc >= average+490:

      #print("Result: 1")
      return 1
    if test_data_x[j][5] == 1:
      return 1
    else:
      #print("Result: 0")

      return 0


In [ ]:
#TODO: Now, run some experiments with your function. Experiment with different values of the parameter theta.
averageP3 = average_finder(training_data_x, training_data_y)
y_hat_banaag_classifier = np.zeros(0)
j = 0

# call banaag_classifier for each value of test data x, return each result to declared array (for loop)
for i in test_data_x:
  comparer = banaag_classifier(test_data_x, averageP3, j)
  #print(comparer)
  
  if comparer == 1:
    y_hat_banaag_classifier = np.insert(y_hat_banaag_classifier, j, 1) #insert are problem, insert at index j?
  else:
    y_hat_banaag_classifier = np.insert(y_hat_banaag_classifier, j, 0)
  j+=1


#

In [ ]:
# TODO: calculate the accuracy of the classifier on the test data with the best
# theta found above and print it.
print(accuracy(test_data_y, y_hat_banaag_classifier))

0.9209302325581395


TODO: Describe in one paragraph your experiments and evaluation. Discuss the overall accuracy your classifier. Did you manage to beat the "majority" classifier? Comment on how easy or hard was to do this. 

  When it came to my experimentation, I ran into the most issues with the synthax of Numpy. Overall, I got the average of the transactions for training_data_x and compared it to each value of test_data_x. If it was greater than by the threshhold I tuned, than it was marked as fraud. The next checker was to see if it was in a high risk country. If it was, I marked it as fraud. Overall, the accuracy increased by six percent, which is pretty solid but can be improved upon. Like I have previously stated, it was quite hard adapting to the python synthax and that was my hardest challenge.


## P4: Implement a logistic regression classifier using sklearn (8 pts)
Implement a logistic regression function using the sklearn library. 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


In [ ]:
# TODO: implement the logistic regression here in a function 
from sklearn.linear_model import LogisticRegression
def logisticregression(training_data_x, training_data_y, test_data_x, test_data_y):
  model = LogisticRegression().fit(training_data_x, training_data_y)

  score = model.score(test_data_x, test_data_y)
  return score

In [ ]:
# TODO: now, run some experiments with it, and measure the accuracy with various parametrizations. In particular, you should run it with and without regularization. 
# In the last line, print the accuracy with the best parameters.
print(logisticregression(training_data_x, training_data_y, test_data_x, test_data_y))

0.987906976744186


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,



TODO: Describe in one paragraph your experiments and evaluation of the Logistic Regression classifier. Consider things such as accuracy, training time, ease of tweaking of the parameters. Compare it with the accuracy of the hand-engineered classifier.

The logistic regression classifier is much easier to impliment and train compared to my hand engineered classifier. When it comes to tweaking the parameters, I only have to limit what data the function gets access too, and tweak which indexes are chosen. Compared to the hand written classifier, this is much easier.

## P5 Bonus: Implement a random forest classifier using sklearn (5 pts)
Implement a random forest classifier using sklearn 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
 # TODO: Implement the random forest classifier here
from sklearn.ensemble import RandomForestClassifier
 
def randomforestfunction(training_data_x, training_data_y, test_data_x, test_data_y):
  model = RandomForestClassifier(max_depth = 3, random_state = 0)
  model.fit(training_data_x, training_data_y)
  prediction = model.score(test_data_x, test_data_y)
  return prediction

In [ ]:
# TODO: Perform some experiments here with different parameters of the random forest classifier. In the last line, print the accuracy with the best parameters.
print(randomforestfunction(training_data_x, training_data_y, test_data_x, test_data_y))

0.9730232558139534


TODO: Describe in one paragraph your experiments and evaluation of the random forest classifier. Consider things such as accuracy, training time, ease of tweaking of the parameters. 

The accuracy seems to be less than the linear regression, but the training time and ease of tweaking of the parameters seem to be more efficient.

## P6 Bonus: Implement an AdaBoost classifer using sklearn (5 pts)

Implement an AdaBoost classifier using sklearn https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

In [ ]:
# TODO: Implement the adaboost classifier here

In [ ]:
# TODO: Perform some experiments here with different parametrizations of the adaboost classifier. In the last line, print the accuracy with the best parameters.

TODO: Describe in one paragraph your experiments and evaluation of the AdaBoost classifier. Consider things such as accuracy, training time, ease of tweaking of the parameters. 

In [ ]:
# SOLUTION
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(training_data_x, training_data_y)
yhat = clf.predict(test_data_x)
acc = accuracy(test_data_y, yhat)
print(f"Accuracy of the AdaBoost classifier {acc}")